##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Prompting with Video

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Video.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

This notebook provides a quick example of how to prompt the Gemini API using a video file. In this case, you'll use a short clip of [Big Buck Bunny](https://peach.blender.org/about/).

**Notice:** This notebook used the foremer Python SDK. Check that more advanced notebook on [Video understanding](./Video_understanding.ipynb) to see how to use the new SDK and discover more video use-cases.

### Install dependencies

In [2]:
%pip install -U "google-genai>=1.0.0"

### Authentication Overview

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

#### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [3]:
from google import genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Upload a video to the Files API

The Gemini API accepts video file formats directly. The File API accepts files under 2GB in size and can store up to 20GB of files per project. Files last for 2 days and cannot be downloaded from the API. For the example, you will use the short film "Big Buck Bunny".

> "Big Buck Bunny" is (c) copyright 2008, Blender Foundation / www.bigbuckbunny.org and [licensed](https://peach.blender.org/about/) under the [Creative Commons Attribution 3.0](http://creativecommons.org/licenses/by/3.0/) License.

Note: You can also [upload your own files](https://github.com/google-gemini/cookbook/blob/main/examples/Upload_files_to_Colab.ipynb) to use.

In [4]:
!wget https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4

--2025-03-30 18:48:41--  https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4
Resolving download.blender.org (download.blender.org)... 172.67.14.163, 104.22.64.163, 104.22.65.163, ...
Connecting to download.blender.org (download.blender.org)|172.67.14.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64657027 (62M) [video/mp4]
Saving to: ‘BigBuckBunny_320x180.mp4.1’

BigBuckBunny_320x18 100%[===================>]  61.66M  94.2MB/s    in 0.7s    

2025-03-30 18:48:42 (94.2 MB/s) - ‘BigBuckBunny_320x180.mp4.1’ saved [64657027/64657027]



In [5]:
video_file_name = "BigBuckBunny_320x180.mp4"

print(f"Uploading file...")
video_file =  client.files.upload(file=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/1f3d6kyolrhe


## Get File

After uploading the file, you can verify the API has successfully received the files by calling `files.get`.

`files.get` lets you see the file uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique.

In [6]:
import time

while video_file.state.name == "PROCESSING":
    print('Waiting for video to be processed.')
    time.sleep(10)
    video_file = client.files.get(name = video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)
print(f'Video processing complete: ' + video_file.uri)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/1f3d6kyolrhe


## Generate Content

After the video has been uploaded, you can make `GenerateContent` requests that reference the File API URI.

In [7]:
# Create the prompt.
prompt = "Describe this video."

# Set the model to Gemini Flash.
MODEL_ID =  "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

# Make the LLM request.
print("Making LLM inference request...")
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[prompt, video_file],
)
print(response.text)

Making LLM inference request...
Here is a description of the video.

The video is a cartoon called Big Buck Bunny. It starts in a forest.  The perspective moves to a cartoon bird in a tree branch. The bird is sleeping, then wakes up and starts singing, but falls off the branch.  The perspective moves to a rabbit sleeping in his den, under a tree.  The rabbit wakes up, walks out, stretches and smells the flowers.  A butterfly flies by and lands on the rabbit's head. He wags his nose, apparently enjoying the flower aroma, but then the butterfly flies away.  The perspective moves to a squirrel up in a tree, then the squirrel and its two friends begin to attack. A branch comes down from the tree.  The view changes to see the rabbit pulling the branch down with a vine. He’s apparently set a trap.

When the tree branch snaps, the video shows a series of wooden spikes shooting up, impaling a peach that was hanging from the tree, and creating a barrier all around it. The view moves back to the

## Delete File

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [8]:
client.files.delete(name=video_file.name)
print(f'Deleted file {video_file.uri}')

Deleted file https://generativelanguage.googleapis.com/v1beta/files/1f3d6kyolrhe


## Next Steps
### Useful API references:

The File API lets you upload a variety of multimodal MIME types, including images, audio, and video formats. The File API handles inputs that can be used to generate content with [`model.generateContent`](https://ai.google.dev/api/rest/v1/models/generateContent) or [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1/models/streamGenerateContent).

* Learn more about the [File API](../quickstarts/File_API.ipynb) with the quickstart.

* Learn more about prompting with [media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length.

### Related examples

Check that more advanced notebook on [Video understanding](./Video_understanding.ipynb) for more use-cases.

Also check those examples videos with the Gemini API to give you more ideas what you can do with them:
* Analyze videos to [classify](../examples/Analyze_a_Video_Classification.ipynb) or [summarize](../examples/Analyze_a_Video_Summarization.ipynb) them
* Have the Gemini API recognize an [historical moment](../examples/Analyze_a_Video_Historic_Event_Recognition.ipynb) and tell you more about it

### Continue your discovery of the Gemini API

If you're not already familiar with it, learn how [tokens are counted](../quickstarts/Counting_Tokens.ipynb). Then check how to use the File API to use [Audio files](../quickstarts/Audio.ipynb) with the Gemini API.
